<a href="https://colab.research.google.com/github/stavco9/datastreaming-final-project/blob/main/Data_Analysis_DeezyMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install DeezyMatch
!pip install thefuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.5 MB/s eta 0:00:00


In [12]:
import os
import tqdm
import time
from DeezyMatch import candidate_ranker, candidate_ranker_init
from thefuzz import process, fuzz

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset"

Mounted at /content/drive
/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset


In [8]:
words_set = []
for i in range(10):
  candidates_file = f"./candidates/dataset_candidates_{i}.txt"
  with open(candidates_file, 'r') as f:
    words_list = f.read().split('\n')
    words_set.append({"words_list": words_list})

In [22]:
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, \
or a wartermelon if possibe. I hope i don't have an i \
don't have an errof in Londaon and in Jerusealm"

In [23]:
for index, words_list in enumerate(words_set):
    time_start_naive = time.time()
    words_set[index]["naive_matches"] = {}
    words_set[index]["naive_matches"]["origin_words"] = []
    words_set[index]["naive_matches"]["matched_words"] = []
    words_set[index]["naive_matches"]["match_scores"] = []
    for word in some_sentance_with_error.split():
      words_set[index]["naive_matches"]["origin_words"].append(word)
      word_match_results = [x for x in process.extract(word, words_list["words_list"], scorer=fuzz.ratio)]
      words_set[index]["naive_matches"]["matched_words"].append(word_match_results[0][0] if len(word_match_results) > 0 else "None")
      words_set[index]["naive_matches"]["match_scores"].append(int(word_match_results[0][1]) / 100 if len(word_match_results) > 0 else 0)
    time_end_naive = time.time()
    words_set[index]["time_naive_sec"] = (time_end_naive - time_start_naive)

In [24]:
for index, words_list in enumerate(words_set):
   print(f"Total time of dataset {index + 1} is {words_list['time_naive_sec']}s")
   print(f"Origin words for dataset {index + 1} are {words_list['naive_matches']['origin_words']}")
   print(f"Matches words for dataset {index + 1} are {words_list['naive_matches']['matched_words']}")
   print(f"Matches scores for dataset {index + 1} are {words_list['naive_matches']['match_scores']}")

Total time of dataset 1 is 4.735710620880127s
Origin words for dataset 1 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone,', 'or', 'a', 'wartermelon', 'if', 'possibe.', 'I', 'hope', 'i', "don't", 'have', 'an', 'i', "don't", 'have', 'an', 'errof', 'in', 'Londaon', 'and', 'in', 'Jerusealm']
Matches words for dataset 1 are ['I', 'Wanna', 'appeal', 'Please', 'or', 'a', 'bananas', 'with', 'a', 'paper', 'and', 'a', 'Lemon', 'or', 'a', 'Waterloo', 'if', 'possible?!', 'I', 'Hope.', 'I', "don't", 'have', 'an', 'I', "don't", 'have', 'an', 'error', 'in', 'London', 'and', 'in', 'Jerusalem']
Matches scores for dataset 1 are [1.0, 1.0, 0.91, 0.92, 1.0, 1.0, 0.86, 1.0, 1.0, 0.89, 1.0, 1.0, 0.91, 1.0, 1.0, 0.74, 1.0, 0.93, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 1.0, 0.92, 1.0, 1.0, 0.89]
Total time of dataset 2 is 4.524561882019043s
Origin words for dataset 2 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', '

In [26]:
for i in range(10):
  # Ranking on-the-fly
  # find candidates from candidate_scenario
  # for queries specified by the `query` argument
  time_start_deezy = time.time()
  ranker = \
      candidate_ranker_init(candidate_scenario=f"./combined/candidates_test_{i}",
                      query=[x.replace("-", " ") for x in some_sentance_with_error.split()],
                      ranking_metric="faiss",
                      selection_threshold=10.,
                      num_candidates=1,
                      search_size=100,
                      output_path=f"ranker_results/test_candidates_deezymatch_on_the_fly_{i}",
                      pretrained_model_path="./models/wikigaz_en_model/wikigaz_en_model.model",
                      pretrained_vocab_path="./models/wikigaz_en_model/wikigaz_en_model.vocab",
                      number_test_rows=len(some_sentance_with_error))

  ranker.rank()
  words_set[index]["deezy_match_df"] = ranker.output
  time_end_deezy = time.time()
  words_set[index]["time_deezy_sec"] = (time_end_deezy - time_start_deezy)

2024-03-23 17:04:59 7cb635edd97d [INFO] read input file: ./combined/candidates_test_0/train_conf.yaml
2024-03-23 17:04:59 7cb635edd97d [INFO] pytorch will use: cuda


FileNotFoundError: [Errno 2] No such file or directory: './combined/candidates_test_0/fwd_id.pt'